In [59]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [78]:
scen={}
scen_name={'':"Reference",'_zero_fossil_2050':"Fossil phase-out",'_high_elec_price':"High electricity price"}


primary_resource_conversion={"Hydrogen":33.3,"Gas":13.1,"Coal":8.06,"Oil":10.89,"Biomass":7.89,"Electricity":1,"MeOH":5.54,"Waste":4.72,"Olefins":5.78,"Ammonia":5.17,"Naphtha":12.47}
primary_resource_color={"Hydrogen":"cyan","Gas":"purple","Coal":"black","Oil":"grey","Biomass":"brown","Electricity":"yellow","MeOH":"red","Waste":"green","Olefins":"darkcyan","Ammonia":"darksalmon","Naphtha":"wheat"}
sector_color={"Steel":"grey","Chemistry":"pink","Cement":"green"}
captured_emissions_sector_color={"Steel":"black","Chemistry":"purple","Cement":"brown"}


for scenario in ['_high_elec_price','','_zero_fossil_2050']:
    res={}
    keys=[]
    for sector in ["Steel","Chemistry","Cement"]:
        with open('Result files/'+sector+'_results'+scenario+'.pickle', 'rb') as f:
            res[sector]=pickle.load(f)
            for key in res[sector].keys():
                if key not in keys:
                    keys.append(key)
        f.close()

    V={}
    for key in keys:
        if key=='tech_tech_type_combinations_parameters':
            t_tt_c={}
            for sector in ["Steel","Chemistry","Cement"]:
                for tech in res[sector][key].keys():
                    if tech not in t_tt_c.keys():
                        t_tt_c[tech]=res[sector][key][tech]
                    else:
                        t_tt_c[tech]+=res[sector][key][tech]
                        t_tt_c[tech]=list(set(t_tt_c[tech]))
            V[key]=t_tt_c
        elif key[:2]=="V_":
            V[key]=res["Steel"][key].reset_index()
            for sector in ["Chemistry","Cement"]:
                V[key]=pd.concat([V[key],res[sector][key].reset_index()],ignore_index=True)
        else:
            pass

    
    resource_inflow=V["V_resource_inflow"].rename(columns={"level_0":"Resource","level_1":"Sector","level_2":"Country","level_3":"Year",0:"Flow"}).sort_values(by="Year").groupby(by=["Resource","Sector","Country","Year"]).sum().reset_index()
    u=resource_inflow.loc[resource_inflow.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])].drop("Resource",axis=1).groupby(by=["Sector","Country","Year"]).sum().reset_index()
    u["Resource"]="Electricity"
    resource_inflow=resource_inflow.loc[~resource_inflow.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])]
    resource_inflow=pd.concat([resource_inflow,u],ignore_index=True)

    resource_outflow=V["V_resource_outflow"].rename(columns={"level_0":"Resource","level_1":"Sector","level_2":"Country","level_3":"Year",0:"Flow"}).sort_values(by="Year").groupby(by=["Resource","Sector","Country","Year"]).sum().reset_index()
    u=resource_outflow.loc[resource_outflow.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])].drop("Resource",axis=1).groupby(by=["Sector","Country","Year"]).sum().reset_index()
    u["Resource"]="Electricity"
    resource_outflow=resource_outflow.loc[~resource_outflow.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])]
    resource_outflow=pd.concat([resource_outflow,u],ignore_index=True)

    resource_flow=V["V_resource_flow"].rename(columns={"level_0":"Resource","level_1":"Sector","level_2":"Country","level_3":"Year",0:"Flow"}).sort_values(by="Year").groupby(by=["Resource","Sector","Country","Year"]).sum().reset_index()
    u=resource_flow.loc[resource_flow.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])].drop("Resource",axis=1).groupby(by=["Sector","Country","Year"]).sum().reset_index()
    u["Resource"]="Electricity"
    resource_flow=resource_flow.loc[~resource_flow.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])]
    resource_flow=pd.concat([resource_flow,u],ignore_index=True)

    resource_tech_inflow=V["V_resource_tech_type_inflow"].rename(columns={"level_0":"Technology","level_1":"Tech_type","level_3":"Resource","level_2":"Sector","level_4":"Country","level_5":"Year",0:"Flow"}).sort_values(by="Year").groupby(by=["Technology","Tech_type","Resource","Sector","Country","Year"]).sum().reset_index()
    u=resource_tech_inflow.loc[resource_tech_inflow.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])].drop("Resource",axis=1).groupby(by=["Technology","Tech_type","Sector","Country","Year"]).sum().reset_index()
    u["Resource"]="Electricity"
    resource_tech_inflow=resource_tech_inflow.loc[~resource_tech_inflow.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])]
    resource_tech_inflow=pd.concat([resource_tech_inflow,u],ignore_index=True)

    resource_tech_outflow=V["V_resource_tech_type_outflow"].rename(columns={"level_0":"Technology","level_1":"Tech_type","level_3":"Resource","level_2":"Sector","level_4":"Country","level_5":"Year",0:"Flow"}).sort_values(by="Year").groupby(by=["Technology","Tech_type","Resource","Sector","Country","Year"]).sum().reset_index()
    u=resource_tech_outflow.loc[resource_tech_outflow.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])].drop("Resource",axis=1).groupby(by=["Technology","Tech_type","Sector","Country","Year"]).sum().reset_index()
    u["Resource"]="Electricity"
    resource_tech_outflow=resource_tech_outflow.loc[~resource_tech_outflow.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])]
    resource_tech_outflow=pd.concat([resource_tech_outflow,u],ignore_index=True)

    tech_usage_flow=resource_tech_outflow.copy()

    elec_consum_fc=V["V_resource_flow"].rename(columns={"level_0":"Resource","level_1":"Sector","level_2":"Country","level_3":"Year",0:"Flow"}).sort_values(by="Year").groupby(by=["Resource","Sector","Country","Year"]).sum().reset_index()
    elec_consum_fc=elec_consum_fc[elec_consum_fc.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])]


    for resource in primary_resource_conversion.keys():
        resource_inflow.loc[resource_inflow.Resource==resource,"Flow"]=primary_resource_conversion[resource]*resource_inflow.loc[resource_inflow.Resource==resource,"Flow"]*1e-6
        resource_outflow.loc[resource_outflow.Resource==resource,"Flow"]=primary_resource_conversion[resource]*resource_outflow.loc[resource_outflow.Resource==resource,"Flow"]*-1e-6
        resource_flow.loc[resource_flow.Resource==resource,"Flow"]=primary_resource_conversion[resource]*resource_flow.loc[resource_flow.Resource==resource,"Flow"]*1e-6
        resource_tech_inflow.loc[resource_tech_inflow.Resource==resource,"Flow"]=primary_resource_conversion[resource]*resource_tech_inflow.loc[resource_tech_inflow.Resource==resource,"Flow"]*1e-6
        resource_tech_outflow.loc[resource_tech_outflow.Resource==resource,"Flow"]=primary_resource_conversion[resource]*resource_tech_outflow.loc[resource_tech_outflow.Resource==resource,"Flow"]*-1e-6

    emissions=V["V_emissions"].rename(columns={"level_0":"Sector","level_1":"Country","level_2":"Year",0:"Emissions"}).sort_values(by="Year")
    captured_emissions=V["V_captured_emissions"].rename(columns={"level_0":"Technology","level_1":"Sector","level_2":"Country","level_3":"Year",0:"Captured_emissions"}).sort_values(by="Year")


    tech_usage_coef=V["V_technology_use_coef"].rename(columns={"level_0":"Technology","level_1":"Tech_type","level_2":"Sector","level_3":"Country","level_4":"Year",0:"Coef"}).sort_values(by="Year")
    tech_usage_capacity_coef=V["V_technology_use_coef_capacity"].rename(columns={"level_0":"Technology","level_1":"Sector","level_2":"Country","level_3":"Year",0:"Coef"}).sort_values(by="Year")

    tech_capacity=V["V_resource_tech_type_capacity"].rename(columns={"level_0":"Technology","level_1":"Tech_type","level_3":"Resource","level_2":"Sector","level_4":"Country","level_5":"Year",0:"Flow_capa"}).sort_values(by="Year")
    added_capa_coef=V["V_added_capacity"].rename(columns={"level_0":"Technology","level_1":"Sector","level_2":"Country","level_3":"Year",0:"Coef_capa"}).sort_values(by="Year")
    remove_capa_coef=V["V_removed_capacity"].rename(columns={"level_0":"Technology","level_1":"Sector","level_2":"Country","level_3":"Year",0:"Coef_capa"}).sort_values(by="Year")
    
    
    df_dict={"resource_inflow":resource_inflow,"resource_outflow":resource_outflow,"resource_flow":resource_flow,
             "resource_tech_inflow":resource_tech_inflow,"resource_tech_outflow":resource_tech_outflow,"tech_usage_flow":tech_usage_flow,
            "emissions":emissions,"captured_emissions":captured_emissions,"elec_consum_fc":elec_consum_fc}
    
    scen[scen_name[scenario]]=df_dict
    
    del V
    del res
    del emissions
    del captured_emissions
    del resource_inflow
    del resource_outflow
    del resource_flow
    del resource_tech_inflow
    del resource_tech_outflow
    del tech_usage_coef
    del tech_usage_capacity_coef
    del tech_capacity
    del added_capa_coef
    del remove_capa_coef
    del elec_consum_fc
    del df_dict


['Ammonia' 'Biomass' 'CEM_I' 'CEM_II' 'CEM_III_VI' 'CO' 'CO2'
 'Calcined_clay' 'Clinker' 'Coal' 'Gas' 'Hydrogen' 'LC3' 'Limestone'
 'MeOH' 'N2' 'Naphtha' 'Oil' 'Olefins' 'Oxygen' 'Steel_primary'
 'Steel_secondary' 'Waste' 'iron_ore' 'scrap_steel']
['Ammonia' 'Biomass' 'CEM_I' 'CEM_II' 'CEM_III_VI' 'CO' 'CO2'
 'Calcined_clay' 'Clinker' 'Coal' 'Gas' 'Hydrogen' 'LC3' 'Limestone'
 'MeOH' 'N2' 'Naphtha' 'Oil' 'Olefins' 'Oxygen' 'Steel_primary'
 'Steel_secondary' 'Waste' 'iron_ore' 'scrap_steel']
['Ammonia' 'Biomass' 'CEM_I' 'CEM_II' 'CEM_III_VI' 'CO' 'CO2'
 'Calcined_clay' 'Clinker' 'Coal' 'Gas' 'Hydrogen' 'LC3' 'Limestone'
 'MeOH' 'N2' 'Naphtha' 'Oil' 'Olefins' 'Oxygen' 'Steel_primary'
 'Steel_secondary' 'Waste' 'iron_ore' 'scrap_steel']


In [61]:
fig =make_subplots(rows=1, cols=3,subplot_titles=list(scen.keys()),shared_yaxes=True)
for i in range(len(scen.keys())):
    emissions_agg=scen[list(scen.keys())[i]]['emissions'].drop(["Sector","Country"],axis=1).groupby(by=["Year"]).sum().reset_index()
    captured_emissions_agg=scen[list(scen.keys())[i]]['captured_emissions'].drop(["Technology","Sector","Country"],axis=1).groupby(by="Year").sum().reset_index()
    showlegend=[True,False,False]
    fig.add_trace(go.Scatter(
        name="Emissions",
        x=emissions_agg.Year, y=emissions_agg.Emissions*1e-6,
        mode='lines',
        line=dict(width=0, color='grey'),
        stackgroup='one',showlegend=showlegend[i]
    ),row=1, col=i+1)
    fig.add_trace(go.Scatter(
        name="Captured emissions",
        x=captured_emissions_agg.Year, y=captured_emissions_agg.Captured_emissions*1e-6,
        mode='lines',
        line=dict(width=0, color='black'),
        stackgroup='one',showlegend=showlegend[i]
    ),row=1, col=i+1)

#     fig.update_xaxes(title_text="Year", row=1, col=i+1)

fig.update_layout(#title='CO2 emissions in EU-5+1',title_x=0.5,
                   yaxis_title='Emissions (MtCO2)')
fig.update_yaxes(range=[0,350])
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.1,
    xanchor="right",
    x=0.75
))
fig.update_annotations(font_size=24)

fig.show()

In [62]:
fig =make_subplots(rows=1, cols=3,subplot_titles=list(scen.keys()),shared_yaxes=True)
for i in range(len(scen.keys())):
    showlegend=[True,False,False]
    
    resource_inflow=scen[list(scen.keys())[i]]['resource_inflow']
    resource_outflow=scen[list(scen.keys())[i]]['resource_outflow']
    
    energy_input=resource_inflow[(resource_inflow.Resource.isin(primary_resource_conversion.keys()))]
    energy_output=resource_outflow[(resource_outflow.Resource.isin(primary_resource_conversion.keys()))]
    energy_input=energy_input.drop(["Sector","Country"],axis=1).groupby(by=["Resource","Year"]).sum().reset_index()
    energy_output=energy_output.drop(["Sector","Country"],axis=1).groupby(by=["Resource","Year"]).sum().reset_index()
    df_merged=pd.concat([energy_input,energy_output],ignore_index=True)


    for resource in primary_resource_conversion.keys():
        if resource not in ["Olefins","Ammonia","Naphtha","CO"]:
            fig.add_trace(go.Scatter(x=energy_input[energy_input.Resource==resource].Year,y=energy_input[energy_input.Resource==resource].Flow,name=resource,
                             line=dict(width=0,color=primary_resource_color[resource]),
                                 fillcolor=primary_resource_color[resource],mode='lines',stackgroup='one',showlegend=showlegend[i])
                         ,row=1, col=i+1)
            fig.add_trace(go.Scatter(x=energy_output[energy_output.Resource==resource].Year,y=energy_output[energy_output.Resource==resource].Flow,name=resource,
                             line=dict(width=0,color=primary_resource_color[resource]),
                                 fillcolor=primary_resource_color[resource],mode='lines',stackgroup='two',showlegend=False)
                          ,row=1, col=i+1)

#     fig.update_xaxes(title_text="Year", row=1, col=i+1)
    
fig.update_layout(#title='Energy flows in EU-5+1', title_x=0.5,
                   yaxis_title='Energy flows (TWh)')
# fig.update_yaxes(range=[-600,1500])
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.1,
    xanchor="right",
    x=1,font = dict(size=20)
))


fig.show()

In [63]:
fig =make_subplots(rows=1, cols=3,subplot_titles=list(scen.keys()),shared_yaxes=True)
for i in range(len(scen.keys())):
    showlegend=[True,False,False]
    resource_flow=scen[list(scen.keys())[i]]['resource_flow']
    r_flow=resource_flow.drop(["Sector","Country"],axis=1).groupby(by=["Resource","Year"]).sum().reset_index()
    for resource in primary_resource_conversion.keys():
        if resource not in ["Olefins","Ammonia","Naphtha","CO","Electricity","Biomass","Waste"]:
            fig.add_trace(go.Scatter(x=r_flow[(r_flow.Resource==resource) & (r_flow.Flow>0)].Year,y=r_flow[(r_flow.Resource==resource) & (r_flow.Flow>0)].Flow,name=resource,
                                 line=dict(width=0,color=primary_resource_color[resource]),
                                 fillcolor=primary_resource_color[resource],hoverinfo="x+y",mode='lines',stackgroup='one',showlegend=showlegend[i]),row=1, col=i+1)
    
#     fig.update_xaxes(title_text="Year", row=1, col=i+1)
    
fig.update_layout(#title='Energy imports in EU-5+1', title_x=0.5,
                   yaxis_title='Energy imports (TWh)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.1,
    xanchor="right",
    x=0.75,font = dict(size=20)
))

In [64]:

hydrogen_techs=["Electrolyser","SMR","SMR_noWGR","SMR_eboiler_noWGR",
                       "eSMR","eSMR_noWGR","eSMR_eboiler_noWGR",
                "Gasification","Gasification_noWGR","Pyrolysis","Naphta_cracking","Naphta_cracking_electrified"]
hydrogen_techs_colors={"Electrolyser":"green","SMR":"black","SMR_noWGR":"black","SMR_eboiler_noWGR":"grey",
                       "eSMR":"yellow","eSMR_noWGR":"orange","eSMR_eboiler_noWGR":"orange",
                "Gasification":"firebrick","Gasification_noWGR":"firebrick","Pyrolysis":"turquoise",
                       "Naphta_cracking":"beige","Naphta_cracking_electrified":"beige"}

tech_name={"Electrolyser":"Electrolyser","SMR":"SMR","SMR_noWGR":"SMR","SMR_eboiler_noWGR":"SMR",
                       "eSMR":"SMR","eSMR_noWGR":"SMR","eSMR_eboiler_noWGR":"SMR",
                "Gasification":"Gasification","Gasification_noWGR":"Gasification","Pyrolysis":"Pyrolysis",
           "Naphta_cracking":"Naphta cracking","Naphta_cracking_electrified":"Naphta cracking"}
tech_name_list={"Electrolyser":["Electrolyser"],"SMR":["SMR","SMR_noWGR","SMR_eboiler_noWGR",
                       "eSMR","eSMR_noWGR","eSMR_eboiler_noWGR"],
                "Gasification":["Gasification","Gasification_noWGR"],"Pyrolysis":["Pyrolysis"],
           "Naphta cracking":["Naphta_cracking","Naphta_cracking_electrified"]}
hydrogen_techs_name_colors={"Electrolyser":"green","SMR":"black",
                "Gasification":"firebrick","Pyrolysis":"turquoise",
                       "Naphta cracking":"beige"}

fig =make_subplots(rows=1, cols=3,subplot_titles=list(scen.keys()),shared_yaxes=True)
for i in range(len(scen.keys())):
    showlegend=[True,False,False]
    tech_usage_flow=scen[list(scen.keys())[i]]['tech_usage_flow']
    tech_usage=tech_usage_flow[(tech_usage_flow.Resource.isin(["Hydrogen"]))]

    hydrogen_tech_usage=tech_usage[tech_usage.Technology.isin(hydrogen_techs)]
    for tech in tech_name_list.keys():
        u=hydrogen_tech_usage[hydrogen_tech_usage.Technology.isin(tech_name_list[tech])]
        u=u[["Year","Flow"]].groupby(by=["Year"]).sum().reset_index()
        fig.add_trace(go.Scatter(x=u.Year,y=u.Flow*1e-6,name=tech,line=dict(width=0,color=hydrogen_techs_name_colors[tech]),
                                 fillcolor=hydrogen_techs_name_colors[tech],hoverinfo="x+y",mode='lines',stackgroup='one',showlegend=showlegend[i]),row=1, col=i+1)

#     fig.update_xaxes(title_text="Year", row=1, col=i+1)

fig.update_layout(#title='Hydrogen production per technology in EU-5+1',  title_x=0.5,
                   yaxis_title='Production (Mt of H2)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.1,
    xanchor="right",
    x=0.9,font = dict(size=20)
))
fig.update_yaxes(range=[0,5.5])
fig.show()


In [65]:
tech_list=['BF-BOF',"DRI-EAF","Electrowinning","EAF"]
steel_techs={"BF-BOF":[['BF-BOF'],['Coal','H2']],
             "BF-BOF w/ Biomass":[['BF-BOF'],['Bio-min','Bio-medium','Bio-max']],             
             "DRI-EAF-H2":[["DRI-EAF"],["H2"]],"DRI-EAF-CH4":[["DRI-EAF"],["CH4"]],
             "Electrowinning":[["Electrowinning"],[0]],"EAF":[["EAF"],[0]]}
steel_techs_colors={"BF-BOF":"Black","BF-BOF w/ Biomass":"brown",
                    "DRI-EAF-H2":"turquoise","DRI-EAF-CH4":'purple',
                    "Electrowinning":"orange","EAF":"yellow"}


fig =make_subplots(rows=1, cols=3,subplot_titles=list(scen.keys()),shared_yaxes=True)
for i in range(len(scen.keys())):
    showlegend=[True,False,False]
    tech_usage_flow=scen[list(scen.keys())[i]]['tech_usage_flow']
    tech_usage=tech_usage_flow[tech_usage_flow.Resource.isin(["Steel_primary","Steel_secondary"])]
    steel_tech_usage=tech_usage[tech_usage.Technology.isin(tech_list)]
    for tech in steel_techs.keys():
            t_list=steel_techs[tech][0]
            tt_list=steel_techs[tech][1]
            u=steel_tech_usage[(steel_tech_usage.Technology.isin(t_list))&(steel_tech_usage.Tech_type.isin(tt_list))]
            u=u[["Year","Flow"]].groupby(by='Year').sum().reset_index()

            fig.add_trace(go.Scatter(x=u.Year,y=u.Flow*1e-6,name=tech,line=dict(width=0,color=steel_techs_colors[tech]), 
                                 fillcolor=steel_techs_colors[tech],
                                 hoverinfo="x+y",mode='lines',stackgroup='one',showlegend=showlegend[i]),row=1, col=i+1)

#     fig.update_xaxes(title_text="Year", row=1, col=i+1)
    
fig.update_layout(#title='Steel production per technology in EU-5+1',title_x=0.5,
                   yaxis_title='Production (Mt of steel)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.1,
    xanchor="right",
    x=1.01,font = dict(size=18)
))
fig.show()

In [66]:
fig =make_subplots(rows=1, cols=3,subplot_titles=list(scen.keys()),shared_yaxes=True)
for i in range(len(scen.keys())):
    showlegend=[True,False,False]
    resource_inflow=scen[list(scen.keys())[i]]['resource_tech_inflow']
    energy_input=resource_inflow[(resource_inflow.Resource.isin(primary_resource_conversion.keys()))]
    energy_input=energy_input.drop(["Sector"],axis=1).groupby(by=["Resource","Country","Year"]).sum().reset_index()
    country_colors={"France":"blue","Germany":"black","Italy":"green","Belgium":"yellow","Spain":"orange","Great Britain":"red"}

    for country in ["France","Germany","Italy","Belgium","Spain","Great Britain"]:
        resource="Electricity"
        fig.add_trace(go.Scatter(x=energy_input[(energy_input.Resource==resource) & (energy_input.Country==country)].Year,
                                 y=energy_input[(energy_input.Resource==resource) & (energy_input.Country==country)].Flow,
                                 name=country,
                                 line=dict(width=0,color=country_colors[country]),
                                     fillcolor=country_colors[country],mode='lines',stackgroup='one',showlegend=showlegend[i]),row=1, col=i+1)
#         fig.update_xaxes(title_text="Year", row=1, col=i+1)
        
fig.update_layout(#title='Electricity consumption per country',title_x=0.5,
                   yaxis_title='Electricity consumption (TWh)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.1,
    xanchor="right",
    x=0.9,font = dict(size=20)
))
# fig.update_yaxes(range=[0,450])
fig.show()

In [67]:
fig =make_subplots(rows=1, cols=3,subplot_titles=list(scen.keys()),shared_yaxes=True)
for i in range(len(scen.keys())):
    showlegend=[True,False,False]
    resource_inflow=scen[list(scen.keys())[i]]['resource_tech_inflow']
    energy_input=resource_inflow[(resource_inflow.Resource.isin(primary_resource_conversion.keys()))]
    energy_input=energy_input[energy_input.Technology!="Electrolyser"]
    energy_input=energy_input.drop(["Sector","Country"],axis=1).groupby(by=["Resource","Year"]).sum().reset_index()
    
    
    resource="Electricity"
    fig.add_trace(go.Scatter(x=energy_input[(energy_input.Resource==resource)].Year,
                             y=energy_input[(energy_input.Resource==resource)].Flow,
                             name='Other electricity consumptions',
                             line=dict(width=0,color="yellow"),
                                 fillcolor="yellow",mode='lines',stackgroup='one',showlegend=showlegend[i]),row=1, col=i+1)
    
    
    energy_input=resource_inflow[(resource_inflow.Resource.isin(primary_resource_conversion.keys()))]
    energy_input=energy_input[energy_input.Technology=="Electrolyser"]
    energy_input=energy_input.drop(["Sector","Country"],axis=1).groupby(by=["Resource","Year"]).sum().reset_index()
    
    
    resource="Electricity"
    fig.add_trace(go.Scatter(x=energy_input[(energy_input.Resource==resource)].Year,
                             y=energy_input[(energy_input.Resource==resource)].Flow,
                             name='Electrolysers',
                             line=dict(width=0,color="turquoise"),
                                 fillcolor="turquoise",mode='lines',stackgroup='one',showlegend=showlegend[i]),row=1, col=i+1)
#     fig.update_xaxes(title_text="Year", row=1, col=i+1)
        
fig.update_layout(#title='Electricity consumption per country',title_x=0.5,
                   yaxis_title='Electricity consumption (TWh)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.1,
    xanchor="right",
    x=0.8,font = dict(size=22)
))
# fig.update_yaxes(range=[0,475])
fig.show()

In [68]:
fig =go.Figure()
color={"Reference":"red","Fossil phase-out":"green","High electricity price":"black"}
for i in range(len(scen.keys())):
    resource_inflow=scen[list(scen.keys())[i]]['resource_tech_inflow']
    energy_input=resource_inflow[(resource_inflow.Resource.isin(primary_resource_conversion.keys()))]
#     energy_input=energy_input[energy_input.Technology!="Electrolyser"]
    energy_input=energy_input.drop(["Sector","Country"],axis=1).groupby(by=["Resource","Year"]).sum().reset_index()
    
    
    resource="Electricity"
    name=list(scen.keys())[i]
    fig.add_trace(go.Scatter(x=energy_input[(energy_input.Resource==resource)].Year,
                             y=energy_input[(energy_input.Resource==resource)].Flow,
                             name=name,line=dict(width=3,color=color[name]),
                                     mode='lines'))

        
fig.update_layout(#title='Electricity consumption per country',title_x=0.5,
                    xaxis_title="Year",
                   yaxis_title='Electricity consumption (TWh)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)



# fig.update_layout(legend=dict(
#     orientation="h",
#     yanchor="bottom",
#     y=1.1,
#     xanchor="right",
#     x=0.9,font = dict(size=20)
# ))
fig.update_yaxes(range=[0,475])
fig.show()

In [70]:
olefins_techs=["Naphta_cracking","Naphta_cracking_electrified","MethanolToOlefins", "OxidativeCouplingOfMethane"]
olefins_techs_colors={"Naphta_cracking":"black","Naphta_cracking_electrified":"grey","MethanolToOlefins":"yellow", "OxidativeCouplingOfMethane":"purple"}
olefins_tech_name={"Naphta cracking":["Naphta_cracking","Naphta_cracking_electrified"],
                        "Methanol to Olefins (MTO)":["MethanolToOlefins"],
                        "Oxidative Coupling of Methane (OCM)":["OxidativeCouplingOfMethane"]}
olefins_techs_name_colors={"Naphta cracking":"black","Methanol to Olefins (MTO)":"yellow", "Oxidative Coupling of Methane (OCM)":"purple"}



fig =make_subplots(rows=1, cols=3,subplot_titles=list(scen.keys()),shared_yaxes=True)
for i in range(len(scen.keys())):
    showlegend=[True,False,False]
    tech_usage_flow=scen[list(scen.keys())[i]]['tech_usage_flow']
    tech_usage=tech_usage_flow[(tech_usage_flow.Resource.isin(["Olefins"]))]
    olefins_tech_usage=tech_usage[tech_usage.Technology.isin(olefins_techs)]
    for tech in olefins_tech_name.keys():
        u=olefins_tech_usage[olefins_tech_usage.Technology.isin(olefins_tech_name[tech])]
        u=u[["Year","Flow"]].groupby(by=["Year"]).sum().reset_index()
        fig.add_trace(go.Scatter(x=u.Year,y=u.Flow*1e-6,name=tech,line=dict(width=0,color=olefins_techs_name_colors[tech]),
                                 fillcolor=olefins_techs_name_colors[tech],hoverinfo="x+y",mode='lines',stackgroup='one',showlegend=showlegend[i]),row=1, col=i+1)
#         fig.update_xaxes(title_text="Year", row=1, col=i+1)

fig.update_layout(#title='Olefins production per technology in EU-5+1',title_x=0.5,
                   yaxis_title='Production (Mt of olefins)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.1,
    xanchor="right",
    x=1,font = dict(size=20)
))
fig.add_annotation(dict(font=dict(color="black",size=28),
                                        x=-0.05,
                                        y=1.15,
                                        showarrow=False,
                                        text="(b)",
                                        textangle=0,
                                        xanchor='left',
                                        xref="paper",
                                        yref="paper"))

fig.show()

In [71]:
naphtha_techs=["Oil_Refining_for_Naphtha","BioNaphtha_making"]
naphtha_techs_colors={"Oil_Refining_for_Naphtha":"black","BioNaphtha_making":"green"}
naphtha_techs_name={"Oil_Refining_for_Naphtha":"Oil refining","BioNaphtha_making":"Bionaphtha"}



fig =make_subplots(rows=1, cols=3,subplot_titles=list(scen.keys()),shared_yaxes=True)
for i in range(len(scen.keys())):
    showlegend=[True,False,False]
    tech_usage_flow=scen[list(scen.keys())[i]]['tech_usage_flow']
    tech_usage=tech_usage_flow[(tech_usage_flow.Resource.isin(["Naphtha"]))]
    naphtha_tech_usage=tech_usage[tech_usage.Technology.isin(naphtha_techs)]
    for tech in naphtha_techs:
        u=naphtha_tech_usage[naphtha_tech_usage.Technology==tech]
        u=u[["Year","Flow"]].groupby(by=["Year"]).sum().reset_index()
        fig.add_trace(go.Scatter(x=u.Year,y=u.Flow*1e-6,name=naphtha_techs_name[tech],line=dict(width=0,color=naphtha_techs_colors[tech]),
                                 fillcolor=naphtha_techs_colors[tech],hoverinfo="x+y",mode='lines',stackgroup='one',showlegend=showlegend[i]),row=1, col=i+1)
#         fig.update_xaxes(title_text="Year", row=1, col=i+1)
        
fig.update_layout(#title='Naphtha production per technology in EU-5+1',title_x=0.5,
                   yaxis_title='Production (Mt of naphtha)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.1,
    xanchor="right",
    x=0.65,font = dict(size=20)
))
fig.add_annotation(dict(font=dict(color="black",size=28),
                                        x=-0.05,
                                        y=1.15,
                                        showarrow=False,
                                        text="(a)",
                                        textangle=0,
                                        xanchor='left',
                                        xref="paper",
                                        yref="paper"))
fig.show()


In [76]:
country_colors={"France":"blue","Germany":"black","Italy":"green","Belgium":"yellow","Spain":"orange","Great Britain":"red"}
elec_power_conv={"Electricity":1,"Electricity_50%_LF":2,"Electricity_25%_LF":4}


fig =make_subplots(rows=1, cols=3,subplot_titles=list(scen.keys()),shared_yaxes=True)
for i in range(len(scen.keys())):
    
    showlegend=[True,False,False]
    consum=scen[list(scen.keys())[i]]['elec_consum_fc']
    consum=consum.drop("Sector",axis=1).groupby(by=["Resource","Country","Year"]).sum().reset_index()
    for resource in ["Electricity","Electricity_50%_LF","Electricity_25%_LF"]:
        consum.loc[consum.Resource==resource,"Flow"]=elec_power_conv[resource]*consum.loc[consum.Resource==resource,"Flow"]*1e-6/8.76
    consum=consum.drop("Resource",axis=1).groupby(by=["Country","Year"]).sum().reset_index() 
    for country in country_colors.keys():
        fig.add_trace(go.Scatter(x=consum[(consum.Country==country)].Year,
                                 y=consum[(consum.Country==country)].Flow,
                                 name=country,
                                 line=dict(width=0,color=country_colors[country]),
                                     fillcolor=country_colors[country],mode='lines',stackgroup='one',showlegend=showlegend[i]),row=1, col=i+1)
fig.update_layout(#title='ammonia production per technology in EU-5+1', title_x=0.5,
#                    xaxis_title='Year',
                   yaxis_title='Power requirement (GW)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.1,
    xanchor="right",
    x=0.8,font = dict(size=16)
))
fig.show()